In this notebook, we'll look at solving coefficient inverse problems for PDEs.
This is a well-studied subject but a problem that often goes ignored is what kind of observational data go into it.
If the observational data are spatially dense then they can be interpolated to the finite element mesh, and you're free to pretend as if the measurements are a nice continuous field.
We'll consider the particular problem of using measurements of the solution $u$ of the Poisson problem

$$-\nabla\cdot k\nabla u = q$$

to estimate the right-hand side $q$.
This can be formulated as the problem of finding a critical point of the functional

$$L =
\underbrace{\frac{1}{2}\int_\Omega\left(\frac{u - u^o}{\sigma}\right)^2dx}_{\text{model-data misfit}} + 
\underbrace{\frac{\alpha^2}{2}\int_\Omega|\nabla q|^2dx}_{\text{regularization}} +
\underbrace{\int_\Omega\left(k\nabla u\cdot\nabla\lambda - q\lambda\right)dx}_{\text{physics constraint}}$$

where $u^o$ are the observational data, and we've introduced a Lagrange multiplier $\lambda$.
This formulation is really nice because the model-data misfit term is an integral that we can easily express in UFL.

But the observational data might be sparse compared to the resolution of the finite element grid, in which case interpolating to a finite element basis might be completely inappropriate.
In that case the model-data misfit has to be written as a finite sum of evaluations at the measurement points $\{x_n\}$:

$$E = \sum_n\frac{|u(x_n) - u^o(x_n)|^2}{2\sigma(x_n)^2}.$$

This might be more correct, but it's much more difficult to express easily in UFL.

In [ ]:
import firedrake
mesh = firedrake.UnitSquareMesh(32, 32)
V = firedrake.FunctionSpace(mesh, family='CG', degree=2)
Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)

The right-hand side $q$ will be a random trigonometric series.

In [ ]:
from firedrake import Constant, cos, sin
import numpy as np
from numpy import pi as π
from numpy import random

seed = 1729
generator = random.default_rng(seed)

degree = 5
x = firedrake.SpatialCoordinate(mesh)

q = firedrake.Function(Q)
for k in range(degree):
    for l in range(int(np.sqrt(degree**2 - k**2))):
        Z = np.sqrt(1 + k**2 + l**2)
        ϕ = 2 * π * (k * x[0] + l * x[1])
        
        A_kl = generator.standard_normal() / Z
        B_kl = generator.standard_normal() / Z
        
        expr = Constant(A_kl) * cos(ϕ) + Constant(B_kl) * sin(ϕ)
        mode = firedrake.interpolate(expr, Q)
        
        q += mode

Ooh pretty

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
axes.set_aspect('equal')
colors = firedrake.tripcolor(q, axes=axes, shading='gouraud')
fig.colorbar(colors);

Now let's generate the true solution of the PDE.

In [ ]:
from firedrake import inner, grad, dx
u = firedrake.Function(V)
J = (0.5 * inner(grad(u), grad(u)) - q * u) * dx
bc = firedrake.DirichletBC(V, 0, 'on_boundary')
F = firedrake.derivative(J, u)
firedrake.solve(F == 0, u, bc)
u_true = u.copy(deepcopy=True)

Sort of pretty I guess I dunno

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
axes.set_aspect('equal')
colors = firedrake.tripcolor(u, axes=axes, cmap='twilight', shading='gouraud')
fig.colorbar(colors);

Now let's make the observational data.

In [ ]:
num_points = 25
δs = np.linspace(-0.5, 2, num_points + 1)
X, Y = np.meshgrid(δs, δs)
xs = np.vstack((X.flatten(), Y.flatten())).T

θ = π / 12
Q = np.array([
    [np.cos(θ), -np.sin(θ)],
    [np.sin(θ), np.cos(θ)]
])

xs = np.array([
    x for x in (xs - np.array([0.5, 0.5])) @ Q
    if (0 <= x[0] <= 1) and (0 <= x[1] <= 1)
])

We'll assume the measurements have a signal-to-noise ratio of 20; you can tweak this.

In [ ]:
U = u_true.dat.data_ro[:]
u_range = U.max() - U.min()
signal_to_noise = 20
σ = u_range / signal_to_noise
ζ = generator.standard_normal(len(xs))
u_obs = np.array(u_true.at(xs)) + σ * ζ

In [ ]:
from mpl_toolkits import mplot3d
fig = plt.figure()
axes = fig.add_subplot(projection='3d')
firedrake.trisurf(u, axes=axes, alpha=0.25, cmap='twilight')
axes.scatter(xs[:, 0], xs[:, 1], u_obs, color='black');

Now we'll create a point cloud object and a function space $Z$ on this point cloud.

In [ ]:
point_cloud = firedrake.VertexOnlyMesh(mesh, xs)
Z = firedrake.FunctionSpace(point_cloud, 'DG', 0)
u_o = firedrake.Function(Z)
u_o.dat.data[:] = u_obs

Next we need to create the operator $E$ that evaluates functions in $V$ on the point cloud.

In [ ]:
ϕ = firedrake.TestFunction(V)
interpolator = firedrake.Interpolator(ϕ, Z)
op2_mat = interpolator.callable()
E = op2_mat.handle
print(f'Type of E: {type(E)}')
print(f'Size of E: {E.getSize()}')

The whole objective functional can be written as

$$J = \frac{1}{2}(Eu - u^o)^*\Sigma^{-1}(Eu - u^o) + R(q) + \langle F(u, q), \lambda\rangle$$

where $\Sigma = \text{diag}([\sigma, \ldots, \sigma])$ is the covariance matrix.
Mathematically, finding a critical point of this functional is a perfectly nice well-defined problem.
But we can't express it directly using UFL yet because of the first term.
We can evaluate the first term and its derivatives, we just get the answer as PETSc data structures instead.

To hack around this difficulty, we'll use a trick from the optimization playbook called *Moreau-Yosida regularization*.
First, we'll introduce an auxiliary variable $v \in V$ and try to find a critical point of

$$J' = \frac{1}{2}(Ev - u^o)^*\Sigma^{-1}(Ev - u^o) + R(q) + \langle F(u, q), \lambda\rangle$$

subject to the constraint that

$$u = v.$$

This is called the *consensus constraint*.
The reformulated problem is completely trivial so at this juncture you might wonder what we've actually accomplished here.
The answer is that we can choose clever ways to enforce the constraint.
We can then split our original, difficult problem into two easy ones.
A blunt approach is the penalty method, which gives the new objective

$$J_\rho = \frac{1}{2}(Ev - u^o)^*\Sigma^{-1}(Ev - u^o) + \frac{\rho}{2}\|u - v\|_M^2 + R(q) + \langle F(u, q), \lambda\rangle.$$

It's up to us to choose what operator $M$ defines the norm we use for the penalty.
This will be a crucial point soon.

##### The $v$ sub-problem.

Suppose we want to minimize $J_\rho$ for $v$ with all the other variables held fixed.
The reduced objective functional is

$$j_\rho(v) = \frac{1}{2}(Ev - u^o)^*\Sigma^{-1}(Ev - u^o) + \frac{\rho}{2}\|u - v\|_M^2,$$

which is convex and quadratic.
The conditions for optimality are a linear system which is easy to assemble using petsc4py.
The optimality system is

$$(E^*\Sigma^{-1}E + \rho M)v = E^*\Sigma^{-1}u^o + \rho Mu.$$

This system is strictly positive-definite.
For large values of $\rho$, we can choose a preconditioner based on whatever would work for $M$.

##### The $u$, $q$, $\lambda$ sub-problem.

Having solved the $v$ sub-problem, we'll then try to find a critical point of the reduced objective

$$j_\rho(u, q, \lambda) = \frac{\rho}{2}\|u - v\|_M^2 + R(q) + \langle F(u, q), \lambda\rangle.$$

This looks like we're solving an inverse problem, only $v$ are now the observational data and we're measuring the misfit in a different metric.
The really important part is that we get to pick $M$.
We could just use the $L^2$-norm, but a smarter choice is

$$\|u - v\|_M^2 = \int_\Omega\left\{(u - v)^2 + \ell^2|\nabla(u - v)|^2\right\}dx$$

where $\ell \approx \text{diameter}(\Omega)$.
This is the norm of the function space $H^1(\Omega)$ where $u$ lives.
Most importantly, **every term of the objective can easily be expressed within UFL.**
We can use pyadjoint and all our old tricks to solve it.

##### The algorithm.

The algorithm we'll use is to alternate between solving the two sub-problems:

1. With $u$, $q$, and $\lambda$ held fixed, solve the $v$ sub-problem using petsc4py.
2. With $v$ held fixed, solve the $u$, $q$, $\lambda$ sub-problem using pyadjoint.

This alternating strategy converges under mild assumptions.

In [ ]:
ψ = firedrake.TrialFunction(V)
ℓ = firedrake.Constant(1.0)
m = (ϕ * ψ + ℓ**2 * inner(grad(ϕ), grad(ψ))) * dx
M = firedrake.assemble(m).M.handle

In [ ]:
ϕ, ψ = firedrake.TestFunction(Z), firedrake.TrialFunction(Z)
I = firedrake.assemble(ϕ * ψ / σ**2 * dx).M.handle

In [ ]:
ρ = 1e-2
K = I.PtAP(E) + ρ * M

The variable `F` will store the right-hand side.
First, we'll sum up the parts coming from the misfit in the $M$-norm between $v$ and $u$ (which is initialized to 0.)

In [ ]:
F = M.createVecRight()
with u.dat.vec_ro as U:
    M.mult(U, F)
    
F *= ρ

Then we'll add the term describing the misfit between $v$ and $u^o$ at the observation points, which uses the evaluation operator $E$.

In [ ]:
G = I.createVecRight()
with u_o.dat.vec_ro as U_o:
    I.mult(U_o, G)
    E.multTransposeAdd(G, F, F)

Now we'll set up and solve a linear system with PETSc.

In [ ]:
from petsc4py import PETSc
ksp = PETSc.KSP().create(firedrake.COMM_WORLD)
ksp.setOperators(K)
ksp.setType(PETSc.KSP.Type.CG)
pc = PETSc.PC().create(firedrake.COMM_WORLD)
pc.setType(PETSc.PC.Type.JACOBI)
pc.setOperators(K)
ksp.setPC(pc)

In [ ]:
v = firedrake.Function(V)
with v.dat.vec as vv:
    ksp.solve(F, vv)

If we plot the results, we get something that looks like we interpolated the observational data.
But there are some string-of-pearls type artifacts that look a lot like what happens when you interpolate sparse point data using inverse distance weighting, which is basically what we just did.

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect('equal')
colors = firedrake.tripcolor(v, cmap='twilight', shading='gouraud', axes=axes)
fig.colorbar(colors);

Now that magic happens...